In [65]:
import pandas as pd

proc_scores = pd.read_csv('~/Documents/processed_score.csv')
raw = pd.read_csv('drugcomb_data_v1.4.csv')

/Users/jarridr/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
block_to_study = raw[['block_id', 'study_name']].drop_duplicates()
proc_scores = proc_scores.merge(block_to_study, on='block_id')
proc_scores

,Unnamed: 0,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,...,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S,drug_row_cid,drug_col_cid,study_name
0,0,1,5-FU,ABT-888,A2058,uM,uM,30.8690,3.865915,6.256584,...,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.8390,3385,11960529,ONEIL
1,1,2,5-FU,ABT-888,A2058,uM,uM,27.4600,8.247403,12.333896,...,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.4300,3385,11960529,ONEIL
2,2,3,5-FU,ABT-888,A2058,uM,uM,29.9010,6.063440,11.660209,...,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.8710,3385,11960529,ONEIL
3,3,4,5-FU,ABT-888,A2058,uM,uM,24.0160,-4.280231,5.145209,...,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.9860,3385,11960529,ONEIL
4,4,5,5-FU,AZD1775,A2058,uM,uM,66.8470,12.284698,15.765467,...,5.126836,0.266027,11.471,25.164,76.501,57.193,30.2120,3385,24856436,ONEIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466028,466028,466255,JNK-IN-7,TARIQUIDAR,KB-ChR-8-5-11,uM,uM,-11.9445,28.991995,23.416838,...,0.033102,0.018344,-13.704,-2.285,-14.149,-9.740,4.0445,57340685,148201,NCATS_MDR_CS
466029,466029,466256,RA-XII,TARIQUIDAR,KB-ChR-8-5-11,uM,uM,12.2270,38.511512,34.275389,...,0.005171,0.255526,-10.940,-3.031,31.219,-6.765,26.1980,14033269,148201,NCATS_MDR_CS
466030,466030,466257,vinorelbine,TARIQUIDAR,KB-ChR-8-5-11,uM,uM,12.7975,33.448102,27.891554,...,0.002872,0.304013,-6.663,-1.943,32.682,-7.087,21.4035,60780,148201,NCATS_MDR_CS
466031,466031,466258,153381-68-1,TARIQUIDAR,KB-ChR-8-5-11,uM,uM,43.5705,46.084903,44.787428,...,0.003753,0.087976,-8.699,-8.227,30.885,56.256,60.4965,3143,148201,NCATS_MDR_CS


In [67]:
pre_row = pd.DataFrame()
pre_col = pd.DataFrame()

proc_scores = proc_scores[proc_scores['study_name'] == 'ALMANAC']

pre_row['cid'] = proc_scores['drug_row_cid']
pre_row['name'] = proc_scores['drug_row']
pre_col['name'] = proc_scores['drug_col']
pre_col['cid'] = proc_scores['drug_col_cid']

all_cid = pd.concat((pre_row, pre_col)).drop_duplicates()

In [74]:
proc_scores['drug_row'].drop_duplicates()

132368    CYTARABINE HYDROCHLORIDE
132369              MERCAPTOPURINE
132370                  MITOMYCINE
132371                 CHEMBL17639
132373                TEMOZOLOMIDE
                    ...           
298918                 RUXOLITINIB
299748                  VANDETANIB
432467                   NSC707389
432664            ARSENIC TRIOXIDE
432794                   NSC256439
Name: drug_row, Length: 103, dtype: object

In [68]:
import numpy as np
cid_to_smiles = np.load('cid_to_smiles_dict.npy', allow_pickle=True).item()

In [69]:
all_cid['smiles'] = all_cid['cid'].apply(lambda x: cid_to_smiles[x])

In [70]:
from rdkit import Chem
from rdkit.Chem.rdmolfiles import MolToSmiles, MolFromSmiles
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem.rdmolops import RemoveHs
import pandas as pd

smiles = []
suppl = Chem.SDMolSupplier('20200925 Export ApexBio (1).sdf')

df_props = ['Name', 'Supplier', 'Supplier ID', 'Compound ID']
salt_remover = SaltRemover()
rows = []
indices = []
for i, mol in enumerate(suppl):
    if mol is None:
        continue
        
    row = {}
    mol = RemoveHs(salt_remover.StripMol(mol))
    row['smiles'] = MolToSmiles(mol, isomericSmiles=False)

    mol_props = set(mol.GetPropNames())
    for prop in df_props:
        try:
            row[prop] = mol.GetProp(prop) if prop in mol_props else None
        except UnicodeDecodeError:
            row[prop] = None

    if mol.HasProp('_Name'):
        row['ID'] = mol.GetProp('_Name')
    
    rows.append(row)
    indices.append(i)
    
apexbio_df = pd.DataFrame(rows, index=indices)

RDKit WARNING: [19:52:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [19:52:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [19:52:02] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [19:52:02] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [16:56:54] ERROR: Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [19:52:03] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [19:52:03] ERROR: Could not sanitize molecule ending on line 151324
RDKit ERROR: [19:52:03] ERROR: Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [19:52:03] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [19:52:03] ERROR: Could not sanitize molecule ending on line 162466
RDKit WARNING: [19:52:03] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [19:52:03] WARNING: not removing hydrogen atom without neighbors


In [8]:
all_cid

,cid,name,smiles
132368,5351180,CYTARABINE HYDROCHLORIDE,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O.Cl
132369,667490,MERCAPTOPURINE,C1=NC2=C(N1)C(=S)N=CN2
132370,5746,MITOMYCINE,CC1=C(C(=O)C2=C(C1=O)N3C[C@H]4[C@@H]([C@@]3([C...
132371,1546,CHEMBL17639,C1C(C(OC1N2C=NC3=C(N=C(N=C32)Cl)N)CO)O
132373,5394,TEMOZOLOMIDE,CN1C(=O)N2C=NC(=C2N=N1)C(=O)N
...,...,...,...
298918,25126798,RUXOLITINIB,C1CCC(C1)[C@@H](CC#N)N2C=C(C=N2)C3=C4C=CNC4=NC=N3
299748,3081361,VANDETANIB,CN1CCC(CC1)COC2=C(C=C3C(=C2)N=CN=C3NC4=C(C=C(C...
432467,54611489,NSC707389,C[C@@H]1CC2CC[C@H]3C(=C)C[C@@H](O3)CC[C@]45C[C...
432664,14888,ARSENIC TRIOXIDE,[O-2].[O-2].[O-2].[As+3].[As+3]


In [71]:
def convert_smiles(x):
    mol = MolFromSmiles(x)
    mol = RemoveHs(salt_remover.StripMol(mol))
    return MolToSmiles(mol, isomericSmiles=False)
    
converted_ours = all_cid
converted_ours['smiles'] = converted_ours['smiles'].apply(convert_smiles)

In [75]:
matching_drugs_df = apexbio_df.merge(converted_ours, on='smiles')
matching_drugs_df['name'].drop_duplicates()

0                                          MITOXANTRONE
1                            DAUNORUBICIN HYDROCHLORIDE
2                                            IFOSFAMIDE
3                                          CHLORAMBUCIL
4                            PROCARBAZINE HYDROCHLORIDE
                            ...                        
70    1-(5-DEOXYPENTOFURANOSYL)-5-FLUORO-4-{[(PENTYL...
71                                          ANASTROZOLE
72                        MECHLORETHAMINE HYDROCHLORIDE
73                                          CABAZITAXEL
74                                                SN-38
Name: name, Length: 64, dtype: object

In [79]:
matching_drugs_df

,smiles,Name,Supplier,Supplier ID,Compound ID,ID,cid,name
0,O=C1c2c(O)ccc(O)c2C(=O)c2c(NCCNCCO)ccc(NCCNCCO...,Mitoxantrone HCL,ApexBio,B2114,UM0118493,,4212,MITOXANTRONE
1,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)CC(O)(C(C...,DNA topoisomerase II inhibitor,ApexBio,B2285,UM0118677,,62770,DAUNORUBICIN HYDROCHLORIDE
2,O=P1(NCCCl)OCCCN1CCCl,Cytostatic agent,ApexBio,A2097,UM0118931,,3690,IFOSFAMIDE
3,O=C(O)CCCc1ccc(N(CCCl)CCCl)cc1,nitrogen mustard alkylating agent,ApexBio,B3716,UM0119017,,2708,CHLORAMBUCIL
4,CNNCc1ccc(C(=O)NC(C)C)cc1,Antineoplastic chemotherapy drug,ApexBio,B1470,UM0119095,,9703,PROCARBAZINE HYDROCHLORIDE
...,...,...,...,...,...,...,...,...
70,CCCCCOC(=O)Nc1nc(=O)n(C2OC(C)C(O)C2O)cc1F,"""DNA, RNA and protein synthesis inhibitor""",ApexBio,A8647,UM0142109,,400633,1-(5-DEOXYPENTOFURANOSYL)-5-FLUORO-4-{[(PENTYL...
71,CC(C)(C#N)c1cc(Cn2cncn2)cc(C(C)(C)C#N)c1,Aromatase inhibitor,ApexBio,B1382,UM0142230,,2187,ANASTROZOLE
72,CN(CCCl)CCCl,DNA synthesis inhibitor,ApexBio,B1785,UM0142367,,5935,MECHLORETHAMINE HYDROCHLORIDE
73,COC1C(=O)C2(C)C(OC)CC3OCC3(OC(C)=O)C2C(OC(=O)c...,Microtubule associated inhibitor,ApexBio,B2157,UM0142424,,9854073,CABAZITAXEL


In [85]:
matching_drugs_df = matching_drugs_df.rename(columns={'Name': 'Description'})
matching_drugs_df

,smiles,Description,Supplier,Supplier ID,Compound ID,ID,cid,name
0,O=C1c2c(O)ccc(O)c2C(=O)c2c(NCCNCCO)ccc(NCCNCCO...,Mitoxantrone HCL,ApexBio,B2114,UM0118493,,4212,MITOXANTRONE
1,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)CC(O)(C(C...,DNA topoisomerase II inhibitor,ApexBio,B2285,UM0118677,,62770,DAUNORUBICIN HYDROCHLORIDE
2,O=P1(NCCCl)OCCCN1CCCl,Cytostatic agent,ApexBio,A2097,UM0118931,,3690,IFOSFAMIDE
3,O=C(O)CCCc1ccc(N(CCCl)CCCl)cc1,nitrogen mustard alkylating agent,ApexBio,B3716,UM0119017,,2708,CHLORAMBUCIL
4,CNNCc1ccc(C(=O)NC(C)C)cc1,Antineoplastic chemotherapy drug,ApexBio,B1470,UM0119095,,9703,PROCARBAZINE HYDROCHLORIDE
...,...,...,...,...,...,...,...,...
70,CCCCCOC(=O)Nc1nc(=O)n(C2OC(C)C(O)C2O)cc1F,"""DNA, RNA and protein synthesis inhibitor""",ApexBio,A8647,UM0142109,,400633,1-(5-DEOXYPENTOFURANOSYL)-5-FLUORO-4-{[(PENTYL...
71,CC(C)(C#N)c1cc(Cn2cncn2)cc(C(C)(C)C#N)c1,Aromatase inhibitor,ApexBio,B1382,UM0142230,,2187,ANASTROZOLE
72,CN(CCCl)CCCl,DNA synthesis inhibitor,ApexBio,B1785,UM0142367,,5935,MECHLORETHAMINE HYDROCHLORIDE
73,COC1C(=O)C2(C)C(OC)CC3OCC3(OC(C)=O)C2C(OC(=O)c...,Microtubule associated inhibitor,ApexBio,B2157,UM0142424,,9854073,CABAZITAXEL


In [91]:
apexbio_and_almanac_drugs = pd.DataFrame()
apexbio_and_almanac_drugs['name'] = proc_scores['drug_row'].drop_duplicates()
apexbio_and_almanac_drugs['Is in ApexBio'] = apexbio_and_almanac_drugs['name'].isin(matching_drugs_df['name'].drop_duplicates())
apexbio_and_almanac_drugs = apexbio_and_almanac_drugs.merge(all_cid, on='name')

apexbio_to_merge_df = matching_drugs_df[['name', 'Description', 'Supplier', 'Supplier ID', 'Compound ID']]
apexbio_and_almanac_drugs = apexbio_and_almanac_drugs.merge(apexbio_to_merge_df, on='name', how='left')
apexbio_and_almanac_drugs = apexbio_and_almanac_drugs.rename(columns={'name': 'Name', 'cid': 'CID', 'smiles': 'Smiles'})
apexbio_and_almanac_drugs = apexbio_and_almanac_drugs.sort_values('Is in ApexBio')

apexbio_and_almanac_drugs.to_csv('k562_drugs_and_in_apexbiolib_status.csv')

In [38]:
all_cid_dups = pd.concat((pre_row, pre_col))

In [39]:
pre=all_cid_dups.groupby(['cid', 'name']).size().to_frame().reset_index()

In [40]:
pre=pre.rename(columns={0: 'num_examples'})

In [41]:
matching_drugs_df = matching_drugs_df.merge(pre, on='cid')

In [15]:
matching_drugs_df = matching_drugs_df.sort_values(by=['num_examples'], ascending=False)

In [42]:
matching_drugs_df = matching_drugs_df.drop(columns=['name_x']).rename(columns={'name_y': 'Name', 'Name': 'Description'})

In [43]:
matching_drugs_df['Name'].drop_duplicates()

0                                          MITOXANTRONE
1                            DAUNORUBICIN HYDROCHLORIDE
2                                            IFOSFAMIDE
3                                          CHLORAMBUCIL
4                            PROCARBAZINE HYDROCHLORIDE
                            ...                        
70    1-(5-DEOXYPENTOFURANOSYL)-5-FLUORO-4-{[(PENTYL...
71                                          ANASTROZOLE
72                        MECHLORETHAMINE HYDROCHLORIDE
73                                          CABAZITAXEL
74                                                SN-38
Name: Name, Length: 64, dtype: object

In [16]:
#matching_drugs_df = matching_drugs_df.drop(columns=['num_examples_y'])
matching_drugs_df = matching_drugs_df.rename(columns={'num_examples_x': 'num_examples'})
cols = matching_drugs_df.columns.to_list()
cols[-2] = 'name'
cols[-1] = 'num_examples'
matching_drugs_df = matching_drugs_df[cols]
matching_drugs_df

KeyError: "['name'] not in index"

In [121]:
matching_drugs_df.to_csv('~/Documents/drug_counts_from_sdf.csv')

In [18]:
pre

,cid,name,num_examples
0,144,5-HYDROXYTRYPTOPHAN (5-HTP),1
1,243,BENZOIC ACID,1
2,247,betaine,2
3,248,TRIMETHYL GLYCINE,2
4,253,BIOTIN (VITAMIN B7),1
...,...,...,...
4145,136257951,NCGC00346436-01,2
4146,136257952,13292-46-1,2
4147,136257953,Ciliobrevin D,2
4148,137031865,SP-2509,2


In [100]:
matching_drugs_df = matching_drugs_df.rename(columns={'Name': 'Description'})

In [64]:
matching_drugs_df['Name'].drop_duplicates()

0                               Mitoxantrone HCL
1                 DNA topoisomerase II inhibitor
2                               Cytostatic agent
3              nitrogen mustard alkylating agent
4               Antineoplastic chemotherapy drug
                         ...                    
69                       Topoisomerase inhibitor
70    "DNA, RNA and protein synthesis inhibitor"
71                           Aromatase inhibitor
73              Microtubule associated inhibitor
74                              TOPO I inhibitor
Name: Name, Length: 72, dtype: object

In [19]:
summ = pd.read_csv('summary_table_v1.4.csv')
raw = pd.read_csv('drugcomb_data_v1.4.csv')

block_to_study = raw[['block_id', 'study_name']].drop_duplicates()
summ = summ.merge(block_to_study, on='block_id')

mono = summ[summ['drug_col'].isna()]
comb = summ[~summ['drug_col'].isna()]

/Users/jarridr/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/jarridr/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
k562_combs = comb[comb['cell_line_name'] == 'K-562']
k562_combs[k562_combs['drug_row'].isin(matching_drugs_df['Name']) & k562_combs['drug_col'].isin(matching_drugs_df['Name'])]

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S,study_name
132369,132370,MERCAPTOPURINE,ANTIBIOTIC AY 22989,K-562,uM,uM,-2.9650,0.669423,-0.251519,-4.418428,-4.495244,0.064536,0.008603,-0.478,-0.035,-2.650,-3.280,-2.4520,ALMANAC
132824,132825,TRETINOIN,THALIDOMIDE,K-562,uM,uM,17.1695,-9.601487,-14.198764,6.114542,-9.443575,4.000000,10.000000,16.719,6.558,21.062,13.277,-6.1075,ALMANAC
133124,133125,CELECOXIB,GEMZAR (LILLY),K-562,uM,uM,6.3665,2.376326,5.610600,-7.679923,-0.941304,0.219596,0.005309,-9.098,1.115,4.715,8.018,14.3495,ALMANAC
133288,133289,ANTIBIOTIC AY 22989,NSC733504,K-562,uM,uM,-3.1680,-1.898111,-4.627620,-6.126912,-5.745244,0.003226,0.004826,2.811,4.828,-4.526,-1.810,-10.8070,ALMANAC
133372,133373,PRALATREXATE,MECHLORETHAMINE HYDROCHLORIDE,K-562,uM,uM,39.7675,-4.074875,-5.977389,-2.959204,-2.624899,0.000226,5.000000,22.496,15.217,51.234,28.301,2.0545,ALMANAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442994,442995,TOPOTECAN HYDROCHLORIDE,ADM HYDROCHLORIDE,K-562,uM,uM,40.7385,-13.993285,-16.935026,-11.781374,-10.067126,0.100000,0.193092,31.370,32.875,38.142,43.335,-23.5065,ALMANAC
443004,443005,GEMZAR (LILLY),ADM HYDROCHLORIDE,K-562,uM,uM,35.9900,-4.689664,-5.383395,-1.160050,-0.293882,0.002981,0.193092,29.861,32.875,37.626,34.354,-26.7460,ALMANAC
443042,443043,DOCETAXEL,ADM HYDROCHLORIDE,K-562,uM,uM,31.2320,-6.520474,-7.996441,-6.521318,-4.961038,0.001494,0.193092,15.529,32.875,28.017,34.447,-17.1720,ALMANAC
443088,443089,NAVELBINE DITARTRATE (TN),ADM HYDROCHLORIDE,K-562,uM,uM,39.4535,-2.827830,-0.772357,0.007005,2.371601,0.012312,0.193092,8.928,32.875,40.629,38.278,-2.3495,ALMANAC


In [23]:
row = pd.DataFrame()
col = pd.DataFrame()

in_biolib = comb[comb['drug_row'].isin(matching_drugs_df['Name']) & comb['drug_col'].isin(matching_drugs_df['Name'])]

row[['cell_line_name', 'drug']] = in_biolib[['cell_line_name', 'drug_col']]
col[['cell_line_name', 'drug']] = in_biolib[['cell_line_name', 'drug_row']]
all_comb_cl_drug = pd.concat((row, col))

In [27]:
def get_uniq_combs(df, study_name=''):
    cell_lines = df['cell_line_name'].to_frame().reset_index().drop(columns=['index']).drop_duplicates()
    if study_name != '':
        df = df[df['study_name'] == study_name]
    
    for_uniq_count = df[['cell_line_name', 'drug_col', 'drug_row']]
    for_uniq_count = for_uniq_count.append(df[['cell_line_name', 'drug_row', 'drug_col']])
    for_uniq_count = for_uniq_count.drop_duplicates()
    
    gb_df = for_uniq_count.groupby('cell_line_name')[['drug_row', 'drug_col']].size().reset_index()
    gb_df = gb_df.rename(columns={0: 'unique_combs_%s' % study_name})
    
    
    zero_val_cell_lines = pd.DataFrame()
    zero_val_cell_lines['cell_line_name'] = cell_lines[~cell_lines['cell_line_name'].isin(gb_df['cell_line_name'])]['cell_line_name']
    zero_val_cell_lines['unique_combs_%s' % study_name] = 0
    
    return gb_df.append(zero_val_cell_lines)

cell_line_counts = all_comb_cl_drug.groupby('cell_line_name')['drug'].nunique().reset_index()
cell_line_counts = cell_line_counts.rename(columns={'drug': 'num_unique_drugs'})
gb = in_biolib.groupby(['cell_line_name', 'drug_row', 'drug_col'])
cell_line_counts = cell_line_counts.merge(get_uniq_combs(in_biolib), on='cell_line_name')
cell_line_counts = cell_line_counts.merge(get_uniq_combs(in_biolib, 'ALMANAC'), on='cell_line_name')
cell_line_counts = cell_line_counts.merge(get_uniq_combs(in_biolib, 'ONEIL'), on='cell_line_name')
cell_line_counts['num_total_experiments'] = in_biolib.groupby('cell_line_name')[['drug_row', 'drug_col']].size().values

In [28]:
cell_line_counts

,cell_line_name,num_unique_drugs,unique_combs_,unique_combs_ALMANAC,unique_combs_ONEIL,num_total_experiments
0,786-0,64,2077,2077,0,2099
1,A2058,11,49,0,49,196
2,A2780,11,49,0,49,196
3,A375,11,49,0,49,196
4,A427,11,49,0,49,196
...,...,...,...,...,...,...
94,UO-31,64,2066,2066,0,2090
95,UWB1289,11,49,0,49,196
96,UWB1289+BRCA1,11,49,0,49,196
97,VCAP,11,49,0,49,196


In [30]:
cell_line_summary = pd.read_csv('cell_line_summary.csv')
cell_line_summary = cell_line_summary.rename(columns={'cell_line': 'cell_line_name'})
cell_line_summary = cell_line_summary.merge(cell_line_counts, on='cell_line_name')
good_zips = comb.groupby('cell_line_name')['synergy_zip'].apply(lambda x: (x > 25.).mean()).reset_index().rename(columns={'synergy_zip': 'zip_above_25_prop'})
cell_line_summary = cell_line_summary.merge(good_zips, on='cell_line_name') 

In [33]:
cell_line_summary[cell_line_summary['cell_line_name'] == 'K-562']

,cell_line_name,cell_line_idx,explained variance,synergy_zip_001,synergy_zip_001_found,num_unique_drugs,unique_combs_,unique_combs_ALMANAC,unique_combs_ONEIL,num_total_experiments,zip_above_25_prop
28,K-562,1086,0.170958,25.789259,9.370543,64,2069,2069,0,2092,0.006335
